In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
test=pd.read_csv("/kaggle/input/ecommerce-analytics-ml-hackathon/test_Yix80N0.csv")
train=pd.read_csv("/kaggle/input/ecommerce-analytics-ml-hackathon/train_8wry4cB.csv")

In [3]:
train.head()

In [4]:
test.head()

# **Data Preprocessing**

In [5]:
#separate date and time in train data
train['Start_Date'] = pd.to_datetime(train['startTime']).dt.date
train['Start_Time'] = pd.to_datetime(train['startTime']).dt.time
train['End_Date'] = pd.to_datetime(train['endTime']).dt.date
train['End_Time'] = pd.to_datetime(train['endTime']).dt.time
#train.head()

#separate date and time in test data
test['Start_Date'] = pd.to_datetime(test['startTime']).dt.date
test['Start_Time'] = pd.to_datetime(test['startTime']).dt.time
test['End_Date'] = pd.to_datetime(test['endTime']).dt.date
test['End_Time'] = pd.to_datetime(test['endTime']).dt.time
#test.head()

In [6]:
#derive day from date in train data
import datetime
from datetime import datetime
 
train.Start_Date=pd.to_datetime(train.Start_Date)
train['Start_Weekday']=train['Start_Date'].dt.weekday

train.End_Date=pd.to_datetime(train.End_Date)
train['End_Weekday']=train['End_Date'].dt.weekday

#train.head()


#derive day from date in test data
test.Start_Date=pd.to_datetime(test.Start_Date)
test['Start_Weekday']=test['Start_Date'].dt.weekday

test.End_Date=pd.to_datetime(test.End_Date)
test['End_Weekday']=test['End_Date'].dt.weekday

#test.head()

In [7]:
#changing object to categorical in train data
list1 =['Start_Time','End_Time','startTime','endTime']

for col in list1:
    train[col] = train[col].astype('str')
    
#train["Start_Time"]= train["Start_Time"].astype(str)     


#changing object to categorical in test data
list2 =['Start_Time','End_Time','startTime','endTime']

for col in list2:
    test[col] = test[col].astype('str')

In [8]:
#Changing datatypes in train data
train["Start_Time"] = train["Start_Time"].apply(pd.to_timedelta)
train["End_Time"] = train["End_Time"].apply(pd.to_timedelta)
train['startTime'] = pd.to_datetime(train['startTime'])
train['endTime'] = pd.to_datetime(train['endTime'])

#Changing datatypes in test data
test["Start_Time"] = test["Start_Time"].apply(pd.to_timedelta)
test["End_Time"] = test["End_Time"].apply(pd.to_timedelta)
test['startTime'] = pd.to_datetime(test['startTime'])
test['endTime'] = pd.to_datetime(test['endTime'])

In [9]:
#adding a column : time spent by a session ID and session day in a particular session in train data
from datetime import datetime
#train['Session_Time'] = train['endTime']-train['startTime']
train['Session_Time']  = (train.endTime-train.startTime)/np.timedelta64(1, 's') #converts to seconds
train['Session_day']= train['End_Date']-train['Start_Date']

#adding a column : time spent by a session ID and session day in a particular session in test data
#test['Session_Time'] = test['endTime']-test['startTime']
test['Session_Time']  = (test.endTime-test.startTime)/ np.timedelta64(1, 's') #converts to seconds
test['Session_day']= test['End_Date']-test['Start_Date']

In [10]:
#dropping starttime and endtime columns in train data 
train=train.drop(['startTime','endTime'], axis = 1) 

#dropping starttime and endtime columns in test data
test=test.drop(['startTime','endTime'], axis = 1) 

In [11]:
#to check if session lasted for more than one day
train[train.Start_Date!=train.End_Date]

In [12]:
#convert timedelta to int in train data
train['Session_day']=train['Session_day'].astype('timedelta64[D]')
train['Session_day']=train['Session_day'].astype('int')

#convert timedelta to int in test data
test['Session_day']=test['Session_day'].astype('timedelta64[D]')
test['Session_day']=test['Session_day'].astype('int')

In [13]:
#checking the length of data where start date is not equal to end date
(len(train[train.Start_Date!=train.End_Date])/len(train))*100

In [14]:
#check days of session where days are less than 1 (session day is the difference between end date and start date)
(len(train[train.Session_day<0])/len(train))*100

In [15]:
#checking rows where start date is greater than end date 
#i.e session start date is greater than session end date which is not possible
(len(train[train.Start_Date>train.End_Date])/len(train))*100

In [16]:
train[train.Start_Date>train.End_Date]

It can't be possible that Session Start date is greater than session end date. 
Example : For session ID u18364, Session start date is 2014-11-25 and session end date is 2014-01-12 which is not possible. 
Clearly it is incorrect data and will hamper in training, hence we will remove this data and as it is only 0.15% of the data so we can drop these values.

In [17]:
#dropping rows where session start date is greater tha session end date
train.drop(train[train.Start_Date>train.End_Date].index, inplace = True)

In [18]:
#dropping column session day as it was only used for analysis in train data
train=train.drop(['Session_day'], axis = 1) 

#dropping column session day as it was only used for analysis in test data
test=test.drop(['Session_day'], axis = 1) 

In [19]:
# making columns : week of the month ; month of the year ; weekday for the end dates in train data
train.Start_Date=pd.to_datetime(train.Start_Date)
train['Week_of_Year']=train['End_Date'].dt.weekofyear
train.End_Date=pd.to_datetime(train.End_Date)
train['Month_of_year']=train['End_Date'].dt.month
#train.head()


# making columns : week of the month ; month of the year ; weekday for the end dates in test data
test.Start_Date=pd.to_datetime(test.Start_Date)
test['Week_of_Year']=test['End_Date'].dt.weekofyear
test.End_Date=pd.to_datetime(test.End_Date)
test['Month_of_year']=test['End_Date'].dt.month

#test.head()

In [20]:
#checking the length of data where start date is not equal to end date
(len(train[train.Start_Weekday!=train.End_Weekday])/len(train))*100

In [21]:
#dropping columns : Start_Date,Start_Time,End_Date,End_Time,Start_Weekday in train data
train=train.drop(['Start_Date','Start_Time','End_Date','End_Time','Start_Weekday'], axis = 1) 

#dropping columns : Start_Date,Start_Time,End_Date,End_Time,Start_Weekday in test data
test=test.drop(['Start_Date','Start_Time','End_Date','End_Time','Start_Weekday'], axis = 1) 

In [22]:
#to check null values
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

In [23]:
#missing_zero_values_table(test)

In [24]:
# Explode/Split column into multiple rows
train1 = pd.DataFrame(train.ProductList.str.split(';').tolist(), index=train.session_id).stack()
train1 = train1.reset_index([0, 'session_id'])
train1.columns = ['session_id', 'ProductList1']
#train1

# Explode/Split column into multiple rows
test1 = pd.DataFrame(test.ProductList.str.split(';').tolist(), index=test.session_id).stack()
test1 = test1.reset_index([0, 'session_id'])
test1.columns = ['session_id', 'ProductList1']
#test1

In [25]:
train1.head()

In [26]:
test1.head()

In [27]:
#concatenating 2 datasets in train data
result_train = train.merge(train1, on='session_id')
#result.head(20)

#concatenating 2 datasets in test data
result_test = test.merge(test1, on='session_id')
#result_test

In [28]:
#dropping ProductList in train data
result_train=result_train.drop(['ProductList'], axis = 1) 

#dropping ProductList in test data
result_test=result_test.drop(['ProductList'], axis = 1) 

In [29]:
result_train[['prd_cat1','prd_cat2','prd_cat3','prd_cat4','drop']]=result_train['ProductList1'].str.rsplit("/", expand=True)
result_test[['prd_cat1','prd_cat2','prd_cat3','prd_cat4','drop']]=result_test['ProductList1'].str.rsplit("/", expand=True)

In [30]:
#dropping column session day as it was only used for analysis in train data
result_train=result_train.drop(['drop','ProductList1'], axis = 1) 

#dropping column session day as it was only used for analysis in test data
result_test=result_test.drop(['drop','ProductList1'], axis = 1) 

In [31]:
#assigning 1 value to female and 2 value to male in train data
result_train.gender[result_train.gender == 'female'] = 0
result_train.gender[result_train.gender == 'male'] = 1

In [32]:
#combining train and test for label encoding
combined_df=result_train.append(result_test)
combined_df.tail()

In [33]:
combined_df.dtypes

In [34]:
#changing object to categorical
## converting Categorical Data into proper forms except account_info
list1 =['prd_cat1','prd_cat2','prd_cat3','prd_cat4']

for col in list1:
    combined_df[col] = combined_df[col].astype('category')

### from category to int labels
for col in list1:
    combined_df[col] = combined_df[col].cat.codes

In [35]:
#changing object to categorical in train data
list3 =['session_id']

for col in list3:
    combined_df[col] = combined_df[col].astype('str')


In [36]:
#splitting test and train again
bool_series = pd.notnull(combined_df["gender"]) 
traindf=combined_df[bool_series] 
#traindf.shape

bool_series1 = pd.isnull(combined_df["gender"]) 
testdf=combined_df[bool_series1] 
#testdf.shape

In [37]:
combined_df.info()

In [38]:
(len([combined_df.Session_Time<0])/len(combined_df))*100

In [39]:
#since we can't drop the rows from test data, ths we will replace the -ve values with mode value
from scipy import stats
stats.mode(combined_df.Session_Time)

#replacing -ve time session with most occurring time value=0.0
combined_df.loc[combined_df.Session_Time<0.00, 'Session_Time'] = 0.00

In [40]:
len([combined_df.Session_Time<0])

In [41]:
combined_df[combined_df.Session_Time<0.00]

In [42]:
#changing object to categorical in train data
list3 =['gender']

for col in list3:
    traindf[col] = traindf[col].astype('int64')


In [43]:
traindf.dtypes

In [44]:
testdf.dtypes

# **Exploratory Data Analysis**

In [45]:
# Total number of rows and columns
traindf.shape

# Rows containing duplicate data
duplicate_rows_df = traindf[traindf.duplicated()]
len(duplicate_rows_df)

In [46]:
# Finding the relations between the variables.
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline 

sns.set(color_codes=True)
plt.figure(figsize=(20,10))
c= traindf.corr()
sns.heatmap(c,cmap="BrBG",annot=True)
c

In [47]:
# Plotting a scatter plot
#fig, ax = plt.subplots(figsize=(10,6))
#ax.scatter(traindf['prd_cat4'], traindf['gender'])
#ax.set_xlabel('prd_cat1')
#ax.set_ylabel('gender')
#plt.show()

In [48]:
import seaborn as sns

sns.set(style="ticks", color_codes=True)
g = sns.pairplot(traindf)
import matplotlib.pyplot as plt
plt.show()

# **Modelling**

# **Using XGBoost**

In [49]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [51]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [52]:
# define dataset
#X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,n_clusters_per_class=2, weights=[0.99], flip_y=0, random_state=7)
X=traindf[['End_Weekday', 'Month_of_year', 'Session_Time', 'Week_of_Year','prd_cat1', 'prd_cat2', 'prd_cat3', 'prd_cat4']]
y=traindf.gender

In [53]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [55]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6, 8, 12, 16],
        'learning_rate' : [0.5,0.1,0.02,0.01,0.005,0.001],
        #'booster':['gbtree','dart'],
        'reg_lambda' : [0.1, 0.5, 1, 2, 3, 4],
        'reg_alpha' : [0.1, 0.5, 1, 2, 3, 4],
        'scale_pos_weight' : [1, 0.5]
        }

In [56]:
#xgb = XGBClassifier(learning_rate=0.02, n_estimators=1000, objective='binary:logistic',silent=True, nthread=1)
xgb = XGBClassifier(n_estimators=1000, objective='binary:logistic',silent=True, nthread=1)

In [57]:
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,y), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, y)
timer(start_time) # timing ends here for "start_time" variable

In [58]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
#results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
#xgb1=XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
 #          colsample_bynode=1, colsample_bytree=1.0, gamma=1.5, gpu_id=-1,
  #          importance_type='gain', interaction_constraints=None,
   #           learning_rate=0.1, max_delta_step=0, max_depth=5,
    #          min_child_weight=10, monotone_constraints=None,
     #         n_estimators=1000, n_jobs=1, nthread=1, num_parallel_tree=1,
      #        objective='binary:logistic', random_state=0, reg_alpha=0,
       #       reg_lambda=1, scale_pos_weight=1, silent=True, subsample=0.8,
        #      tree_method=None, validate_parameters=False, verbosity=None)

#0.90 F1 score
#0.871666666666667

In [ ]:
#xgb2=XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
 #             colsample_bynode=1, colsample_bytree=0.8, gamma=2, gpu_id=-1,
  #            importance_type='gain', interaction_constraints=None,
   #           learning_rate=0.02, max_delta_step=0, max_depth=16,
    #          min_child_weight=5, monotone_constraints=None,
     #         n_estimators=1000, n_jobs=1, nthread=1, num_parallel_tree=1,
      #        objective='binary:logistic', random_state=0, reg_alpha=2,
       #       reg_lambda=0.5, scale_pos_weight=0.5, silent=True, subsample=0.8,
        #      tree_method=None, validate_parameters=False, verbosity=None)
#0.88  F1 score
#0.858333333333333

In [59]:
xgb3=XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
            colsample_bynode=1, colsample_bytree=1.0, gamma=1.5, gpu_id=-1,
            importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=10, monotone_constraints=None,
              n_estimators=1000, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, silent=True, subsample=0.8,
              tree_method=None, validate_parameters=False, verbosity=None)
        
#0.90 f1score      
#0.874444444444444

In [ ]:
#xgb4 = XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
 #           colsample_bynode=1, colsample_bytree=1.0, gamma=1.5, gpu_id=-1,
  #          importance_type='gain', interaction_constraints=None,
   #           learning_rate=0.1, max_delta_step=0, max_depth=5,
    #          min_child_weight=10, monotone_constraints=None,
     #         n_estimators=1000, n_jobs=1, nthread=1, num_parallel_tree=1,
      #        objective='binary:hinge', random_state=0, reg_alpha=0,
       #       reg_lambda=1, scale_pos_weight=1, silent=True, subsample=0.8,
        #      tree_method=None, validate_parameters=False, verbosity=None)
        
#0.866666666666667

In [ ]:
#xgb5=XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
 #           colsample_bynode=1, colsample_bytree=1.0, gamma=1.5, gpu_id=-1,
  #          importance_type='gain', interaction_constraints=None,
   #           learning_rate=0.1, max_delta_step=0, max_depth=5,
    #          min_child_weight=10, monotone_constraints=None,
     #       n_estimators=2000, n_jobs=1, nthread=1, num_parallel_tree=1,
      #        objective='binary:logistic', random_state=0, reg_alpha=0,
       #     reg_lambda=1, scale_pos_weight=1, silent=True, subsample=0.8,
        #      tree_method=None, validate_parameters=False, verbosity=None)

#0.872777777777778
#F1 0.90

In [60]:
xgb3.fit(X_train, y_train,eval_set=[(X_test, y_test)],eval_metric=['rmsle','auc','map'],verbose=True)

In [61]:
preds = xgb3.predict(X_test)

In [62]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, preds))

In [63]:
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(y_test, preds))

# **Submission Code**

In [64]:
testdf.columns

In [65]:
test=testdf[['End_Weekday', 'Month_of_year', 'Session_Time', 'Week_of_Year','prd_cat1', 'prd_cat2', 'prd_cat3', 'prd_cat4']]

In [66]:
pred_data = xgb3.predict(test)

In [67]:
pred_data = pd.DataFrame({'session_id':testdf['session_id'],'gender':pred_data})
sub = pred_data.groupby(['session_id']).max()
sub.gender[sub.gender == 0] = 'female'
sub.gender[sub.gender == 1] = 'male'
sub.to_csv('Submission_XGBoost_final.csv',index=True)

# **Using LightGBM**

In [ ]:
#import lightgbm as lgb
#from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
#from sklearn.metrics import roc_auc_score
#from sklearn.model_selection import StratifiedKFold

In [ ]:
# define dataset
#X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,n_clusters_per_class=2, weights=[0.99], flip_y=0, random_state=7)
#X=traindf[['End_Weekday', 'Month_of_year', 'Session_Time', 'Week_of_Year','prd_cat1', 'prd_cat2', 'prd_cat3', 'prd_cat4']]
#y=traindf.gender

In [ ]:
#from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
#traindf = lgb.Dataset(X_train, label=y_train, feature_name=X_train.columns)

In [ ]:
#validationdf=lgb.Dataset(X_test, label=y_test, feature_name=X_test.columns)

In [ ]:
#def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
 #                      model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
  #                     do_probabilities = False):
   # gs = GridSearchCV(
    #    estimator=model,
     #   param_grid=param_grid, 
      #  cv=cv, 
       # n_jobs=-1, 
        #scoring=scoring_fit,
        #verbose=2
    #)
    #fitted_model = gs.fit(X_train_data, y_train_data)
    
    #if do_probabilities:
     # pred = fitted_model.predict_proba(X_test_data)
    #else:
     # pred = fitted_model.predict(X_test_data)
    
    #return fitted_model, pred

In [ ]:
#model = lgb.LGBMClassifier()
#param_grid = {
 #   'n_estimators': [400, 700, 1000],
  #  'colsample_bytree': [0.7, 0.8],
   # 'max_depth': [15,20,25],
    #'num_leaves': [50, 100, 200],
    #'reg_alpha': [1.1, 1.2, 1.3],
 #   'reg_lambda': [1.1, 1.2, 1.3],
 #   'min_split_gain': [0.3, 0.4],
 #   'subsample': [0.7, 0.8, 0.9],
 #   'subsample_freq': [20]
#}

#model, pred = algorithm_pipeline(X_train, X_test, y_train, y_test, model, 
#                                 param_grid, cv=5, scoring_fit='accuracy')
#print(model.best_score_)
#print(model.best_params_)

In [ ]:
#{'colsample_bytree': 0.7, 'max_depth': 20, 'min_split_gain': 0.3, 'n_estimators': 400, 'num_leaves': 200, 'reg_alpha': 1.1, 
 #'reg_lambda': 1.2, 'subsample': 0.9, 'subsample_freq': 20}

In [ ]:
#lgbm=lgb.LGBMClassifier(colsample_bytree=0.7 , max_depth=20, min_split_gain=0.3 , n_estimators=400 ,num_leaves=200,
 #                      reg_alpha=1.1, reg_lambda=1.2, subsample=0.9, subsample_freq=20, boosting='dart',
  #                      metric = ['auc', 'binary_logloss'], objective='binary',early_stopping_round=100)

In [ ]:
#lgbm.fit(X_train, y_train,eval_set=[(X_test, y_test)],eval_metric=['binary_logloss','auc'],verbose=True)

In [ ]:
#preds = lgbm.predict(X_test)

In [ ]:
#from imblearn.metrics import classification_report_imbalanced
#print(classification_report_imbalanced(y_test, preds))

In [ ]:
#pred_data = lgbm.predict(test)

In [ ]:
#pred_data = pd.DataFrame({'session_id':testdf['session_id'],'gender':pred_data})
#sub = pred_data.groupby(['session_id']).max()
#sub.gender[sub.gender == 0] = 'female'
#sub.gender[sub.gender == 1] = 'male'
#sub.to_csv('Submission_LightGBM_gbdt.csv',index=True)

# **Stacked Classifier**

In [ ]:
#from sklearn.pipeline import make_pipeline
#from sklearn.ensemble import StackingClassifier
#from sklearn.linear_model import LogisticRegression
#from catboost import CatBoostClassifier

In [ ]:
#model_st = CatBoostClassifier(leaf_estimation_iterations= 20,depth=8,random_seed= 42,l2_leaf_reg= 3,
 #              iterations=2000,learning_rate= 0.05,logging_level='Silent',loss_function='Logloss',bootstrap_type='Bayesian',
  #             bagging_temperature=1,eval_metric='F1') #use_best_model=True,# subsample=0.66,#eval_set=(X_test,y_test))

In [ ]:
#import xgboost as xgb
#xgb_st=xgb.XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
 #           colsample_bynode=1, colsample_bytree=1.0, gamma=1.5, gpu_id=-1,
  #          importance_type='gain', interaction_constraints=None,
   #           learning_rate=0.1, max_delta_step=0, max_depth=5,
    #          min_child_weight=10, monotone_constraints=None,
     #         n_estimators=1000, n_jobs=1, nthread=1, num_parallel_tree=1,
      #        objective='binary:logistic', random_state=0, reg_alpha=0,
       #       reg_lambda=1, scale_pos_weight=1, silent=True, subsample=0.8,
        #      tree_method=None, validate_parameters=False, verbosity=1)

In [ ]:
#lgbm_st=lgb.LGBMClassifier(colsample_bytree=0.7 , max_depth=20, min_split_gain=0.3 , n_estimators=400 ,num_leaves=200,
 #                      reg_alpha=1.1, reg_lambda=1.2, subsample=0.9, subsample_freq=20, boosting='dart',
  #                      metric = ['auc', 'binary_logloss'], objective='binary',early_stopping_round=100)

In [ ]:
#estimators = [
 #   ('cat',model_st),
  #  ('xgb',xgb_st),
   # ('lgbm',lgbm_st)
#]

In [ ]:
#clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [ ]:
#clf.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
#preds=clf.predict(X_test)

In [ ]:
#from imblearn.metrics import classification_report_imbalanced
#print(classification_report_imbalanced(y_test, preds))

In [ ]:
#pred_data = clf.predict(test)

In [ ]:
#pred_data = pd.DataFrame({'session_id':testdf['session_id'],'gender':pred_data})
#sub = pred_data.groupby(['session_id']).max()
#sub.gender[sub.gender == 0] = 'female'
#sub.gender[sub.gender == 1] = 'male'
#sub.to_csv('Submission_stacked.csv',index=True)